# [Day 19: Beacon Scanner](https://adventofcode.com/2021/day/19)

In [1]:
import collections as cl
import functools as ft
import itertools as it
import re

## Part 1

In [2]:
example_data = [
    "--- scanner 0 ---",
    "404,-588,-901",
    "528,-643,409",
    "-838,591,734",
    "390,-675,-793",
    "-537,-823,-458",
    "-485,-357,347",
    "-345,-311,381",
    "-661,-816,-575",
    "-876,649,763",
    "-618,-824,-621",
    "553,345,-567",
    "474,580,667",
    "-447,-329,318",
    "-584,868,-557",
    "544,-627,-890",
    "564,392,-477",
    "455,729,728",
    "-892,524,684",
    "-689,845,-530",
    "423,-701,434",
    "7,-33,-71",
    "630,319,-379",
    "443,580,662",
    "-789,900,-551",
    "459,-707,401",
    "",
    "--- scanner 1 ---",
    "686,422,578",
    "605,423,415",
    "515,917,-361",
    "-336,658,858",
    "95,138,22",
    "-476,619,847",
    "-340,-569,-846",
    "567,-361,727",
    "-460,603,-452",
    "669,-402,600",
    "729,430,532",
    "-500,-761,534",
    "-322,571,750",
    "-466,-666,-811",
    "-429,-592,574",
    "-355,545,-477",
    "703,-491,-529",
    "-328,-685,520",
    "413,935,-424",
    "-391,539,-444",
    "586,-435,557",
    "-364,-763,-893",
    "807,-499,-711",
    "755,-354,-619",
    "553,889,-390",
    "",
    "--- scanner 2 ---",
    "649,640,665",
    "682,-795,504",
    "-784,533,-524",
    "-644,584,-595",
    "-588,-843,648",
    "-30,6,44",
    "-674,560,763",
    "500,723,-460",
    "609,671,-379",
    "-555,-800,653",
    "-675,-892,-343",
    "697,-426,-610",
    "578,704,681",
    "493,664,-388",
    "-671,-858,530",
    "-667,343,800",
    "571,-461,-707",
    "-138,-166,112",
    "-889,563,-600",
    "646,-828,498",
    "640,759,510",
    "-630,509,768",
    "-681,-892,-333",
    "673,-379,-804",
    "-742,-814,-386",
    "577,-820,562",
    "",
    "--- scanner 3 ---",
    "-589,542,597",
    "605,-692,669",
    "-500,565,-823",
    "-660,373,557",
    "-458,-679,-417",
    "-488,449,543",
    "-626,468,-788",
    "338,-750,-386",
    "528,-832,-391",
    "562,-778,733",
    "-938,-730,414",
    "543,643,-506",
    "-524,371,-870",
    "407,773,750",
    "-104,29,83",
    "378,-903,-323",
    "-778,-728,485",
    "426,699,580",
    "-438,-605,-362",
    "-469,-447,-387",
    "509,732,623",
    "647,635,-688",
    "-868,-804,481",
    "614,-800,639",
    "595,780,-596",
    "",
    "--- scanner 4 ---",
    "727,592,562",
    "-293,-554,779",
    "441,611,-461",
    "-714,465,-776",
    "-743,427,-804",
    "-660,-479,-426",
    "832,-632,460",
    "927,-485,-438",
    "408,393,-506",
    "466,436,-512",
    "110,16,151",
    "-258,-428,682",
    "-393,719,612",
    "-211,-452,876",
    "808,-476,-593",
    "-575,615,604",
    "-485,667,467",
    "-680,325,-822",
    "-627,-443,-432",
    "872,-547,-609",
    "833,512,582",
    "807,604,487",
    "839,-516,451",
    "891,-625,532",
    "-652,-548,-490",
    "30,-46,-14",
]

In [3]:
def parse_data(input_data):
    re_header = re.compile(r"^\s*\-\-\-\s*scanner\s*(?P<nr>\d+)\s*\-\-\-\s*$", re.IGNORECASE)
    data = {}
    for line in input_data:
        line = line.strip()
        if mt_header := re_header.match(line):
            nr = int(mt_header["nr"])
            data[nr] = []
        elif line:
            data[nr].append(tuple([int(c) for c in line.split(",")]))
    return data

In [4]:
@ft.cache
def rotate(x, y, z, i):
    rotations = (
        (x, y, z),
        (x, -z, y),
        (x, -y, -z),
        (x, z, -y),
        (-x, y, -z),
        (-x, z, y),
        (-x, -y, z),
        (-x, -z, -y),
        (y, z, x),
        (y, -x, z),
        (y, -z, -x),
        (y, x, -z),
        (-y, z, -x),
        (-y, x, z),
        (-y, -z, x),
        (-y, -x, -z),
        (z, x, y),
        (z, -y, x),
        (z, -x, -y),
        (z, y, -x),
        (-z, x, -y),
        (-z, y, x),
        (-z, -x, y),
        (-z, -y, -x),
    )
    return rotations[i]

In [5]:
def remap_scanners(scanner_data):
    # use first scanner as base / anchor
    base_scanner_nr = list(scanner_data.keys())[0]
    known_scanners = { base_scanner_nr: (0, 0, 0) }
    beacons_per_scanner = { base_scanner_nr: scanner_data[base_scanner_nr] }
    all_beacons = set(scanner_data[base_scanner_nr])

    # match all other scanners against known scanners
    unknown_scanners = cl.deque(list(scanner_data.keys())[1:])
    while unknown_scanners:
        # work on next unknown scanner
        found_match = False
        current_scanner = unknown_scanners.popleft()
        for known_scanner in list(known_scanners.keys()):
            # compare it against all known scanners
            for rotation in range(24):
                # Using every possible rotation,
                distances = cl.defaultdict(int)
                for kx, ky, kz in beacons_per_scanner[known_scanner]:
                    # get all distances between all beacons (known and unknown)
                    for sx, sy, sz in scanner_data[current_scanner]:
                        sx, sy, sz = rotate(sx, sy, sz, rotation)
                        deltas = kx-sx, ky-sy, kz-sz
                        distances[deltas] += 1
                # if we have a match: we have at least 12 items with the same x, y and z distance
                for (dx, dy, dz), count in distances.items():
                    if count >= 12:
                        # match: remap scanner and its beacons
                        found_match = True
                        known_scanners[current_scanner] = (dx, dy, dz)
                        beacons_per_scanner[current_scanner] = []
                        for sx, sy, sz in scanner_data[current_scanner]:
                            sx, sy, sz = rotate(sx, sy, sz, rotation)
                            beacon = (sx+dx, sy+dy, sz+dz)
                            beacons_per_scanner[current_scanner].append(beacon)
                            all_beacons.add(beacon)
                        break
                if found_match:
                    # exit rotation loop
                    break
            if found_match:
                break  # exit known_scanner loop
        else:
            # no early exit -> no match with a known scanner
            # keep current scanner and repeat later (when we know more scanners)
            unknown_scanners.append(current_scanner)

    return known_scanners, beacons_per_scanner, all_beacons

In [6]:
%%time
known_scanners, beacons_per_scanner, all_beacons = remap_scanners(parse_data(example_data))

Wall time: 124 ms


In [7]:
print(f"Check part 1: {len(all_beacons) == 79}")

Check part 1: True


In [8]:
with open(r"..\data\Day 19 input.txt", "r") as fh_in:
    input_data = [row.strip() for row in fh_in.readlines()]
print(f"Input check: {len(input_data) == 1060}")

Input check: True


In [9]:
%%time
known_scanners, beacons_per_scanner, all_beacons = remap_scanners(parse_data(input_data))
print(f"Answer part 1: {len(all_beacons)}")

Answer part 1: 472
Wall time: 13.6 s


## Part 2

In [10]:
def manhattan_distance(p1, p2):
    return sum([abs(left-right) for left, right in zip(p1, p2)])

In [11]:
%%time

# slightly inefficient, but don't optimize prematurely...
max_distance = 0
for scanner1, scanner2 in it.permutations(known_scanners.keys(), 2):
    distance = manhattan_distance(known_scanners[scanner1], known_scanners[scanner2])
    if distance > max_distance:
        max_distance = distance

print(f"Answer part 2: {max_distance}")

Answer part 2: 12092
Wall time: 2 ms
